# Merge to Master Sheet

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path(".")
RAW = BASE/"data/raw"
INTERIM = BASE/"data/interim"

read_opts = dict(dtype=str, keep_default_na=True, na_values=["", "NA", "NaN"], low_memory=False)

nih = pd.read_csv(RAW/"NIH_final.csv", **read_opts)
td  = pd.read_csv(RAW/"Temp_Disc_Final.csv", **read_opts)
ph = pd.read_csv(INTERIM/"HBN_pheno_with_diagnosis.csv", **read_opts)

print(nih.shape, td.shape, ph.shape)

(3879, 64) (2506, 115) (3373, 174)


In [3]:
def detect_eid_col(df):
    """
    Try to find the participant ID column even if the header is odd.
    Rules:
      - exact 'EID'
      - header contains 'EID' (e.g., 'NIH_final,EID', 'Subject EID')
      - OR a column whose values look like NDAR IDs (start with 'NDAR')
    Returns the column name or None.
    """
    cols = list(df.columns)

    # header contains 'EID' anywhere (e.g., 'NIH_final,EID')
    for c in cols:
        if "EID" in str(c).upper():
            return c

    # look for values that look like NDAR IDs
    def looks_like_ndar(s):
        s = s.astype(str).str.upper()
        return s.str.startswith("NDAR").mean()  # fraction that start with NDAR

    best_c, best_score = None, 0.0
    for c in cols:
        try:
            score = looks_like_ndar(df[c])
            if score > best_score:
                best_c, best_score = c, score
        except Exception:
            pass

    if best_score >= 0.5:   # at least half the column looks like NDAR IDs
        return best_c

    return None


def normalize_id(s: pd.Series) -> pd.Series:
    return (s.astype(str)
              .str.replace("\ufeff","", regex=False)
              .str.upper().str.strip()
              .str.replace(r"[^A-Z0-9]", "", regex=True))  # drop dashes/spaces

for name, df in [("NIH", nih), ("TempDisc", td), ("PhenoDx", ph)]:
    eid_col = detect_eid_col(df)
    if eid_col is None:
        raise KeyError(f"{name}: could not find an EID column. Columns = {df.columns.tolist()}")
    if eid_col != "EID":
        df.rename(columns={eid_col: "EID"}, inplace=True)
    df["_EID"] = normalize_id(df["EID"])
    print(f"{name}: detected EID column = {eid_col!r}, unique IDs = {df['_EID'].nunique()}")

NIH: detected EID column = 'NIH_final,EID', unique IDs = 3879
TempDisc: detected EID column = 'assessment Temp_Disc_Final,EID', unique IDs = 2506
PhenoDx: detected EID column = 'EID', unique IDs = 3373


In [4]:
for name, df in [("NIH", nih), ("TD", td), ("PhenoDx", ph)]:
    dups = df.columns[df.columns.duplicated()].tolist()
    print(f"{name} duplicate cols:", dups)
    if "_EID" in df.columns:
        print(f"{name} count('_EID') =", (df.columns == "_EID").sum())

NIH duplicate cols: []
NIH count('_EID') = 1
TD duplicate cols: []
TD count('_EID') = 1
PhenoDx duplicate cols: []
PhenoDx count('_EID') = 1


In [9]:
set_nih = set(nih["_EID"].dropna())
set_td  = set(td["_EID"].dropna())
set_ph  = set(ph["_EID"].dropna())

print("Overlap NIH ∩ TD:", len(set_nih & set_td))
print("Overlap NIH ∩ PhenoDx:", len(set_nih & set_ph))
print("Overlap TD  ∩ PhenoDx:", len(set_td  & set_ph))
print("Overlap all three    :", len(set_nih & set_td & set_ph))

# Build a working master table (inner join keeps only participants present in both tables)
# left (Temp Discounting) – keep one _EID
td_left = td.loc[:, ["_EID"] + [c for c in td.columns if c not in ("EID","_EID")]]

# right (NIH) – keep one _EID
nih_right = nih.loc[:, ["_EID"] + [c for c in nih.columns if c not in ("EID","_EID")]]

# demographics from PhenoDx
ph_slim = ph.loc[:, ["_EID","Sex","Age"]].drop_duplicates("_EID")

master = (td_left.merge(nih_right, on="_EID", how="inner", suffixes=("_td","_nih"))
                 .merge(ph_slim,   on="_EID", how="inner"))

INTERIM = Path("data/interim")
INTERIM.mkdir(parents=True, exist_ok=True)

out_csv = INTERIM / "NIH_TempDisc_pheno_diagnosis.csv"
master.to_csv(out_csv, index=False)
print(f"Saved: {out_csv}  ->  shape={master.shape}")

Overlap NIH ∩ TD: 2430
Overlap NIH ∩ PhenoDx: 2923
Overlap TD  ∩ PhenoDx: 2097
Overlap all three    : 2076
Saved: data/interim/NIH_TempDisc_pheno_diagnosis.csv  ->  shape=(2076, 180)


# Check Missingness

In [17]:
master_rawview = master.copy()  # preserve a version before any pd.to_numeric(..., errors="coerce")

In [18]:
# Columns present in each raw table (excluding the join key)
cols_td  = [c for c in td_left.columns  if c != "_EID"]
cols_nih = [c for c in nih_right.columns if c != "_EID"]
cols_ph  = [c for c in ["Sex","Age"] if c in ph_slim.columns]

# Restrict to columns that actually exist in master
cols_td  = [c for c in cols_td  if c in master_rawview.columns]
cols_nih = [c for c in cols_nih if c in master_rawview.columns]
cols_ph  = [c for c in cols_ph  if c in master_rawview.columns]

print(f"From TD:  {len(cols_td)} cols")
print(f"From NIH: {len(cols_nih)} cols")
print(f"From Ph:  {len(cols_ph)} cols")

From TD:  113 cols
From NIH: 62 cols
From Ph:  2 cols


In [21]:
def compare_missing(master_df, raw_df, cols, src_name):
    """
    For each column in `cols`, compare the NaN mask in master_df vs raw_df, row-aligned by _EID.
    Returns a report DataFrame with counts of matches/mismatches.
    """
    out = []
    base = master_df[["_EID"]].drop_duplicates()
    for col in cols:
        if col not in raw_df.columns or col not in master_df.columns:
            continue
        tmp = (base.merge(master_df[["_EID", col]], on="_EID", how="left")
                   .merge(raw_df[["_EID", col]],   on="_EID", how="left", suffixes=("_m","_raw")))
        m_na  = tmp[f"{col}_m"].isna()
        r_na  = tmp[f"{col}_raw"].isna()
        row = {
            "source": src_name,
            "column": col,
            "N_compared": len(tmp),
            "master_na": int(m_na.sum()),
            "raw_na": int(r_na.sum()),
            "match_rate": float((m_na == r_na).mean()),
            "master_na_but_raw_not": int((m_na & ~r_na).sum()),
            "raw_na_but_master_not": int((~m_na & r_na).sum()),
        }
        out.append(row)
    rep = pd.DataFrame(out).sort_values(
        ["master_na_but_raw_not","raw_na_but_master_not","match_rate"], ascending=[False, False, True]
    )
    return rep

In [22]:
rep_td  = compare_missing(master_rawview, td_left,  cols_td,  "TD")
rep_nih = compare_missing(master_rawview, nih_right, cols_nih, "NIH")
rep_ph  = compare_missing(master_rawview, ph_slim,   cols_ph,  "PhenoDx")

print("TD columns — worst mismatches first")
display(rep_td.head(10).style.hide(axis="index"))

print("NIH columns — worst mismatches first")
display(rep_nih.head(10).style.hide(axis="index"))

print("Pheno/Demographics — mismatches")
display(rep_ph.style.hide(axis="index"))

TD columns — worst mismatches first


source,column,N_compared,master_na,raw_na,match_rate,master_na_but_raw_not,raw_na_but_master_not
TD,"assessment Temp_Disc_Final,Administration",2076,0,0,1.000000,0,0
TD,"assessment Temp_Disc_Final,Comment_ID",2076,2076,2076,1.000000,0,0
TD,"assessment Temp_Disc_Final,Data_entry",2076,0,0,1.000000,0,0
TD,"assessment Temp_Disc_Final,Days_Baseline",2076,1224,1224,1.000000,0,0
TD,"assessment Temp_Disc_Final,PSCID",2076,2076,2076,1.000000,0,0
TD,"assessment Temp_Disc_Final,START_DATE",2076,0,0,1.000000,0,0
TD,"assessment Temp_Disc_Final,Season",2076,0,0,1.000000,0,0
TD,"assessment Temp_Disc_Final,Site",2076,0,0,1.000000,0,0
TD,"assessment Temp_Disc_Final,Study",2076,0,0,1.000000,0,0
TD,"assessment Temp_Disc_Final,Temp_Disc_run1_ed50",2076,46,46,1.000000,0,0


NIH columns — worst mismatches first


source,column,N_compared,master_na,raw_na,match_rate,master_na_but_raw_not,raw_na_but_master_not
NIH,"NIH_final,Administration",2076,0,0,1.000000,0,0
NIH,"NIH_final,Age",2076,13,13,1.000000,0,0
NIH,"NIH_final,Comment_ID",2076,2076,2076,1.000000,0,0
NIH,"NIH_final,Data_entry",2076,0,0,1.000000,0,0
NIH,"NIH_final,Days_Baseline",2076,0,0,1.000000,0,0
NIH,"NIH_final,Handedness",2076,2,2,1.000000,0,0
NIH,"NIH_final,NIH_Card_Sort_Age_Corr_Stnd",2076,8,8,1.000000,0,0
NIH,"NIH_final,NIH_Card_Sort_Computed_Score",2076,7,7,1.000000,0,0
NIH,"NIH_final,NIH_Card_Sort_Inst_Breakoff",2076,7,7,1.000000,0,0
NIH,"NIH_final,NIH_Card_Sort_Inst_Status",2076,7,7,1.000000,0,0


Pheno/Demographics — mismatches


source,column,N_compared,master_na,raw_na,match_rate,master_na_but_raw_not,raw_na_but_master_not
PhenoDx,Sex,2076,1,1,1.000000,0,0
PhenoDx,Age,2076,1,1,1.000000,0,0


In [25]:
RESULTS = Path("results"); RESULTS.mkdir(exist_ok=True)

# Column-wise missing %
missing_pct = master.isna().mean()

# Drop exactly-100%-missing columns
drop_cols = missing_pct[missing_pct == 1.0].index.tolist()
master_nz = master.drop(columns=drop_cols)
print(f"Dropped {len(drop_cols)} columns with 100% missing.")

# Save a record of what was dropped
pd.Series(drop_cols, name="dropped_100pct_missing").to_csv(
    RESULTS/"dropped_100pct_missing.txt", index=False
)

Dropped 10 columns with 100% missing.


In [27]:
# Recompute on the non-empty columns
miss = master_nz.isna().mean().sort_values(ascending=False)  # fraction missing
miss_df = pd.DataFrame({"column": miss.index, "missing_pct": miss.values})

# 10% bins: [0–10), [10–20), …, [90–100]
bins   = np.arange(0, 1.01, 0.1)  # 0.0, 0.1, …, 1.0
labels = [f"{int(b*100)}–{int((b+0.1)*100)}%" for b in bins[:-1]]
miss_df["bin"] = pd.cut(miss_df["missing_pct"], bins=bins, labels=labels, include_lowest=True, right=False)

# Ranked table (per column)
display(
    miss_df
      .assign(missing_pct=lambda d: (d["missing_pct"]*100).round(1))
      .rename(columns={"missing_pct":"missing_%", "bin":"missing_bin"})
      .head(30)  # top 30 most-missing columns
)

# Grouped table (bin -> count, average %, column names)
grouped = (miss_df
           .groupby("bin", observed=True, sort=True)
           .agg(n_cols=("column","size"),
                avg_missing=("missing_pct","mean"),
                columns=("column", lambda s: ", ".join(s.tolist())))
           .reset_index()
           .rename(columns={"bin":"missing_bin"}))

grouped["avg_missing_%"] = (grouped["avg_missing"]*100).round(1)
grouped = grouped.drop(columns=["avg_missing"])

display(grouped)

,column,missing_%,missing_bin
0,"NIH_final,NIH_Picture_Vocab_Rescored",100.0,90–100%
1,"NIH_final,NIH_Processing_Rescored",98.9,90–100%
2,"NIH_final,NIH_Picture_Seq_Inst_Breakoff",98.7,90–100%
3,"NIH_final,NIH_Picture_Seq_Age_Corr_Stnd",98.7,90–100%
4,"NIH_final,NIH_Picture_Seq_Computed_Score",98.7,90–100%
5,"NIH_final,NIH_Picture_Seq_Uncorr_Stnd",98.7,90–100%
6,"NIH_final,NIH_Picture_Seq_Raw",98.7,90–100%
7,"NIH_final,NIH_Picture_Seq_Itm_Cnt",98.7,90–100%
8,"NIH_final,NIH_Picture_Seq_Inst_Status",98.7,90–100%
9,"NIH_final,NIH_Picture_Vocab_Inst_Status",98.6,90–100%


,missing_bin,n_cols,columns,avg_missing_%
0,0–10%,152,"NIH_final,Sex, assessment Temp_Disc_Final,Temp...",2.1
1,50–60%,1,"assessment Temp_Disc_Final,Days_Baseline",59.0
2,80–90%,1,"NIH_final,NIH_List_Sort_Rescored",82.7
3,90–100%,16,"NIH_final,NIH_Picture_Vocab_Rescored, NIH_fina...",98.6


## Pheno EDA

In [13]:
import re

# heuristics for TD and NIH features
td_candidates = [c for c in num_cols if re.search(r"(logk|auc|beta|k_?|delay|reward)", c, re.I)]
nih_candidates = [c for c in num_cols if re.search(r"(flanker|dccs|pattern|psm|list|processing|attention|flexibility|speed)", c, re.I)]

print("TD candidates (first 12):", td_candidates[:12])
print("NIH candidates (first 12):", nih_candidates[:12])

summary_cols = ["Age", "Sex"] + td_candidates[:6] + nih_candidates[:6]
summary_cols = [c for c in summary_cols if c in master.columns]

display(master[summary_cols].describe(include="all").T.style.set_caption("Descriptives (subset)"))

TD candidates (first 12): []
NIH candidates (first 12): []


,count,unique,top,freq
Age,2075,1968,11.341318,4
Sex,2075,2,0.0,1363


## NIH Picture Sequence Memory Measures

In [30]:
if isinstance(master.columns, pd.MultiIndex):
    master = master.copy()
    master.columns = ["|".join(map(str, tup)).strip("|") for tup in master.columns]

# Make sure everything is string for regex matching
master = master.rename(columns={c: str(c) for c in master.columns})

def find_cols(df, patterns):
    """
    Return a sorted list of columns whose names match ANY of the regex
    patterns (case-insensitive).
    """
    rx = re.compile("|".join(patterns), re.I)
    return sorted([c for c in df.columns if rx.search(c)])

def pct_missing_table(df, cols):
    """
    Percent missing per column and quick group summaries.
    """
    out = (df[cols].isna().mean().sort_values(ascending=False) * 100).round(1)
    tbl = out.reset_index().rename(columns={"index":"column", 0:"pct_missing"})
    # Group summaries
    overall_pct_missing = out.mean().round(1)
    rows_any_missing = (df[cols].isna().any(axis=1).mean() * 100).round(1)
    rows_complete = (df[cols].notna().all(axis=1).mean() * 100).round(1)
    summary = {
        "overall_avg_pct_missing_across_cols": overall_pct_missing,
        "rows_with_any_missing_%": rows_any_missing,
        "rows_complete_%": rows_complete,
        "n_rows": len(df),
        "n_cols": len(cols)
    }
    return tbl, summary

# ----------------------------------------------------------------
# Column patterns
# Picture Sequence Memory (NIH) – covers many naming styles
picseq_patterns = [
    r"NIH.*Picture.*Seq",       # "NIH_Picture_Seq_*"
    r"Picture.*Seq",            # "Picture Sequence ..."
    r"Picture\s*Sequence",      # with space
    r"^NIH_Picture_Seq",        # exact prefix
]

print("Found Picture Sequence cols:", len(picseq_cols))
for c in picseq_cols: print("  •", c)

Found Picture Sequence cols: 8
  • NIH_final,NIH_Picture_Seq_Age_Corr_Stnd
  • NIH_final,NIH_Picture_Seq_Computed_Score
  • NIH_final,NIH_Picture_Seq_Inst_Breakoff
  • NIH_final,NIH_Picture_Seq_Inst_Status
  • NIH_final,NIH_Picture_Seq_Itm_Cnt
  • NIH_final,NIH_Picture_Seq_Raw
  • NIH_final,NIH_Picture_Seq_Rescored
  • NIH_final,NIH_Picture_Seq_Uncorr_Stnd


KeyError: 'No column ends with pattern: Temp_Disc_run1_k'